# Cleanup Resources

Each notebook has a resource cleanup section that is designed to remove just the resources it created.

This notebook is designed to remove all resources by the type created.  By being a general cleanup utility, you can skip sections that you want to remain in use.  This notebook also has a section to delete the GCS bucket created in the `00 - Initial Setup` notebook.

**Resources**
- AI Platform: https://googleapis.dev/python/aiplatform/latest/index.html
- BigQuery: https://googleapis.dev/python/bigquery/latest/index.html
- GCS: https://googleapis.dev/python/storage/latest/client.html

---

## Setup

Parameters:

In [14]:
PROJECT_ID = 'statmike-mlops'
REGION = 'us-central1'
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

AI Platform Client Setup:

In [15]:
from google.cloud import aiplatform
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

---

## BigQuery
- Delete the dataset `digits`

Make a client connection to BigQuery

In [2]:
from google.cloud import bigquery
bq = bigquery.Client(project=PROJECT_ID)

Delete the dataset `digits`

In [4]:
bq.delete_dataset(PROJECT_ID+'.digits',delete_contents=True,not_found_ok=True)

---

## AI Platform (Unified) > Endpoints

- Undeploy models
- Delete endpoints

In [53]:
clients['endpoint'] = aiplatform.gapic.EndpointServiceClient(client_options=client_options)

In [61]:
#clients['endpoint'].list_endpoints(parent=PARENT)

In [56]:
for x in clients['endpoint'].list_endpoints(parent=PARENT):
    print("endpoint name: ",x.display_name)
    for y in x.deployed_models:
        print("deployed model name: ",y.display_name)
        clients['endpoint'].undeploy_model(endpoint=x.name, deployed_model_id=y.id)

endpoint name:  ENDPOINT_AIP_DIGITS_20210409142218
deployed model name:  DEPLOYED_AIP_DIGITS_20210409142218
endpoint name:  ENDPOINT_AIP_DIGITS_20210408205703
deployed model name:  DEPLOYED_AIP_DIGITS_20210408205703
endpoint name:  DIGITS2
deployed model name:  KERAS-DIGITS
endpoint name:  DIGITS
deployed model name:  BQML-DIGITS


In [60]:
for x in clients['endpoint'].list_endpoints(parent=PARENT):
    print("endpoint name: ",x.display_name)
    clients['endpoint'].delete_endpoint(name=x.name)

endpoint name:  DIGITS
endpoint name:  DIGITS2
endpoint name:  ENDPOINT_AIP_DIGITS_20210408205703
endpoint name:  ENDPOINT_AIP_DIGITS_20210409142218


## AI Platform (Unified) > Models
- Delete uploaded models

In [19]:
clients['model'] = aiplatform.gapic.ModelServiceClient(client_options=client_options)

In [62]:
for x in clients['model'].list_models(parent=PARENT):
    print(x.display_name)
    clients['model'].delete_model(name=x.name)

AIP_DIGITS_20210409142218
AIP_DIGITS_20210408205703
KERAS-DIGITS
BQML-DIGITS


## AI Platform (Unified) > Training > Custom Job
- Delete custom training jobs

In [21]:
clients['job'] = aiplatform.gapic.JobServiceClient(client_options=client_options)

In [63]:
for x in clients['job'].list_custom_jobs(parent=PARENT):
    print(x.display_name)
    clients['job'].delete_custom_job(name=x.name)

AIP_DIGITS_20210409142218-custom-job
AIP_DIGITS_20210408205703


## AI Platform (Unified) > Training > Training Pipeline
- Delete training pipelines

In [25]:
clients['pipeline'] = aiplatform.gapic.PipelineServiceClient(client_options=client_options)

In [64]:
for x in clients['pipeline'].list_training_pipelines(parent=PARENT):
    print(x.display_name)
    clients['pipeline'].delete_training_pipeline(name=x.name)

AIP_DIGITS_20210409142218


## Storage Bucket Removal

- delete contents and storage bucket `statmike-models`

In [36]:
from google.cloud import storage
gcs = storage.Client()

In [37]:
bucket = gcs.get_bucket(PROJECT_ID)

In [38]:
bucket.delete(force=True)